# 客户标签体系构建

## 5.1 客户标签体系构建
通过客户交易数据的深度挖掘，构建全面的客户标签体系，能够对客户进行精准画像，从而在产品营销中准确高效地定位和筛选目标客户，实现精准营销。在本环节，我们将结合结构化数据和非结构化文本数据，从交易属性、消费偏好、行为特征三大维度构建客户标签体系。我们的标签体系一共包含150个标签，客户标签体系框架如图5.1所示：

<center><img src=".\Pics\Pic5_1.jpg" width=600 height = 600 alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图5.1：客户标签体系</font></center>

根据具体标签的构建方式，可以将客户标签分为以下四类：
+ 事实类标签：可以直接从客户交易记录中进行统计和计算的标签。例如网购消费、餐饮消费、商旅消费等。
+ 规则类标签：在事实类标签的基础上，结合人工经验，对客户的某项指标进行的计算或归类。例如RFM标签、是否休眠客户、是否有高端消费等。
+ 预测类标签：原始数据中不能直接提取，需要借助模型进行预测的标签。例如客户价值等级。
+ 文本类标签：从客户交易记录的文本中提取的关键词也可用于描述客户偏好，将此部分关键词作为文本类标签。例如彩票、儿童、孕妇、基金等。

从原始数据`data`中提取客户标签，保存客户标签表`user_features`中。每一行代表一个客户，一共有7630行，每个客户需要提取150个标签，分别存放在150列中。如图5.2所示：
<center><img src=".\Pics\Pic5_2.jpg" width=1000 height = 600 alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图5.2：标签提取示意图</font></center>

客户标签体系的构建流程如图5.3所示：
<center><img src=".\Pics\Pic5_3.jpg"width=700 height=200 alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图5.3：客户标签体系构建流程</font></center>

在标签构建完成后，我们选取了两个典型客户，从交易时间、交易次数、交易金额和交易附言内容等几个维度进行了用户画像的刻画和分析。例如客户的词云图、交易流入流出的对比分析、交易描述的分布和不同客户行为的排序及分析等，进一步加深对典型客户的理解。

## 5.2 事实类标签的计算（交易次数和交易总额）
我们首先构建事实类标签。事实类标签是指可以直接从客户交易记录中进行统计和计算的标签，本项目中的40个事实类标签如下：
<center><font size="2.5px" face="微软雅黑" color="666666">表5.1：事实类标签</font></center>
<center><img src=".\Pics\Table5_1.jpg" width="750" height="250" alt="cursor" align=center /></center>

接下来我们分别对每个标签进行提取和计算。对单个客户来说，他会产生金额流出记录和金额流入记录。其数量相加在一定程度上表示了客户的收入能力和消费能力，因此在这里我们提取其交易次数和总额作为客户的标签。两个标签的计算方法如下：

<center><font size="2.5px" face="微软雅黑" color="666666">表5.2：交易次数和总额的计算方法</font></center>

|英文|中文|计算方法|
|:---:|:---:|:---:|
|total_transactions_cnt|交易次数|每个客户交易记录的条数|
|total_transactions_amt|交易总额|每个客户交易的金额总和(金额取绝对值)|


Pandas的`groupby()`函数将DataFrame数据依照某属性进行分组，语法如下`DataFrame.groupby(by=None,axis=0)`
+ `by`：用来表示确定`groupby()`函数的分组依据，如`by='user_id'`则表示根据`user_id`进行分组。
+ `axis`：0表示按行分组，1表示按列分组，默认为0。

`groupby()`函数返回的是一个GroupBy对象，它实际上没有进行任何计算。对GroupBy对象选取列，然后使用聚合函数进行运算。比如，使用`size()`方法查看各组的个数、使用`mean()`函数可以求平均值，使用`sum()`函数可以求和。

>实训任务
>+ 新建`user_features`表，用于存放客户的标签，表的索引为`user_id`，此步骤已默认给出。每提取一个标签，就在`user_features`表中新增一列进行存放。
>+ 请使用`groupby()`函数对`data`根据`user_id`进行分组，计算客户的交易次数和交易总金额，并将其结果分别存储到`user_features`表的`total_transactions_cnt`和`total_transactions_amt`列中。
>  + 要求：计算交易流水总额时需要将每笔交易金额取绝对值(使用`abs()`函数)，再使用`sum()`方法进行求和。

In [ ]:
import pandas as pd

user_features = pd.DataFrame(index = data["user_id"].unique())

# 计算客户的交易次数
user_features['total_transactions_cnt'] = data.groupby('user_id').size()

# 计算客户的交易总金额
user_features['total_transactions_amt'] = abs(data.payment).groupby(data['user_id']).sum()

print(user_features.head())

## 5.3 事实类标签的计算（转账次数和转账总额）
接下来我们提取客户的转账类标签。使用关键字`转账`进行匹配，将客户有转帐行为的流水记录提取出来，针对转账记录我们再对其进行分析，计算客户的转账次数、转账总金额和转账平均金额。三个客户标签的计算方法如下：

<center><font size="2.5px" face="微软雅黑" color="666666">表5.3：转账类标签的计算方法</font></center>

|英文|中文|计算方法|
|:---:|:---:|:---:|
|transfer_cnt|转账次数|每个客户转账记录的条数|
|transfer_amt|转账总金额|每个客户转账记录的金额总和|
|transfer_mean|转账平均金额|$\frac{转账总金额}{转账次数}$|

对某一列Series对象进行关键词匹配(即判断文本中有没有这个词)，首先要将此列的文本数据转换为Pandas中的string类型(`pandas.core.strings`)，接着使用`contains()`函数带入关键词进行匹配。语法如下：`Series.str.contains(pattern, case=True)`
+ `pattern` : 待匹配的关键词，可以是字符，也可以是正则表达式。
+ `case` : 区分大小写，默认为`True`。

>实训任务
>+ 请从`data`中提取包含转账行为的行，保存在DataFrame对象`transfer`中，此步骤已默认给出。
>+ 请使用`groupby()`函数对`transfer`对象进行分组，计算客户的转账次数、转账总金额和转账平均金额，并将其结果分别存储到`user_features`表的`transfer_cnt`、`transfer_amt`和`transfer_mean`列中。
>  + 要求：计算转账总金额和平均金额时需要将每笔交易金额取绝对值(使用`abs()`函数)，再使用`sum()`/`mean()`方法计算总和/平均值。

In [ ]:
import pandas as pd

# 提取包含转账行为的记录
transfer = data[data['describe'].str.contains('转账')]

# 计算客户的转账次数
user_features["transfer_cnt"] = transfer.groupby('user_id').size()

# 计算客户的转账总金额
user_features["transfer_amt"] = abs(transfer.payment).groupby(transfer['user_id']).sum()

# 计算客户的转账平均金额
user_features["transfer_mean"] = abs(transfer.payment).groupby(transfer['user_id']).mean()

print(user_features.head())

## 5.4 事实类标签的计算（其他）
接下来我们对客户产生消费的记录进行提取，对于这一行为的分析，可以更好的帮我们理解客户的消费水平和消费能力。

消费行为是金额流出的过程，我们首先选取交易金额大于0的交易记录。但是在金额流出的行为中，"转账"、"提现"、"转入"、"还款"等行为并不属于消费，我们需要将其剔除。在此部分我们同样使用关键词匹配的方法对客户交易数据进行筛选，接着提取出客户的单次最大消费金额、消费订单比例等标签。两个客户标签的计算方法如下：

<center><font size="2.5px" face="微软雅黑" color="666666">表5.4：消费类标签的计算方法</font></center>

|英文|中文|计算方法|
|:---:|:---:|:---:|
|max_consume_amt|单次最大消费金额|每个客户单次消费的最大值|
|consume_order_ratio|消费订单比例|$\frac{每个客户的消费订单总数}{每个客户的交易次数}$|


>实训任务
>+ 请选取交易金额大于0的交易记录，保存在DataFrame变量`consume_info`中。接着剔除"转账"、"提现"、"转入"、"还款"等记录，将结果保存在DataFrame变量`consume`中，此步骤已题目已默认完成。
>+ 请使用`groupby()`函数对`consume`对象进行分组，计算单次最大消费金额、消费订单比例，并将其结果分别存储到`user_features`表的`max_consume_amt`和`consume_order_ratio`列中。
>  + 提示：使用`groupby()`函数根据`user_id`进行聚合；对GroupBy对象使用`max()`方法查看各组中的最大值。
> + 最终共提取出40个事实类标签，由于提取方法类似，题目并没有将标签的提取完全展示。40个标签的抽取为事先完成，在之后的题目中直接可调用`user_features`表。

通过以上过程，我们已经了解了如何从原始数据中抽取特征，40个事实类标签已抽取并保存在`user_features`表中。

In [ ]:
import pandas as pd

# 选取交易金额大于0的交易记录
consume_info = data[data.payment > 0]

# 选取包含消费行为的记录
consume = consume_info[~consume_info["describe"].str.contains('转账|提现|转入|还款')]

# 单次最大消费金额
user_features["max_consume_amt"] = consume.groupby('user_id')['payment'].max()

# 消费订单比例
user_features["consume_order_ratio"] = consume.groupby('user_id').size()  / user_features['total_transactions_cnt']

#################### 其余事实类标签提取展示如下 ########################

# 提取商旅消费的记录
train_bus = consume[consume["describe"].str.contains("车票|火车票|机票")]
travel_hotel = consume[consume["describe"].str.contains("住宿|宾馆|酒店|携程|去哪")]
business_travel = pd.concat([train_bus,travel_hotel])

# 商旅消费次数
user_features["business_travel_cnt"] = business_travel.groupby("user_id").size()

# 商旅消费总金额
user_features["business_travel_amt"] = business_travel.payment.groupby(business_travel["user_id"]).sum()

# 商旅消费平均金额
user_features["business_travel_avg_amt"] = business_travel.payment.groupby(business_travel["user_id"]).mean()

# 提取公共事业缴费的记录
public = consume_info[consume_info["describe"].str.contains('水费|电费|燃气费')]

# 公共事业缴费总额
user_features["public_pay_amt"] = public.groupby("user_id")['payment'].sum()

print(user_features.head())

## 5.5 规则类标签的计算（有无高端消费）
在完成事实类标签后，我们开始提取规则类标签。规则类标签是在事实类标签的基础上，结合人工经验，对客户的某项指标进行的计算或归类，如定义高端消费需要人为规定阈值。在项目中，我们提取的规则类标签包括有无高端消费、是否休眠、RFM类等9个标签，如下所示：

<center><font size="2.5px" face="微软雅黑" color="666666">表5.5：规则类标签</font></center>
<center><img src=".\Pics\Table5_5.jpg" width="750" height="250" alt="cursor" align=center /></center>

随着客户可支配收入水平的升高，客户的消费偏好会发生变化。对某些中、高档商品的购买和消费量会增加，对低档消费品的需求减少，因此在这里我们对是否有高端消费进行定义，作为客户的一个标签。

在前面我们已经计算了客户的最大消费金额，我们取所有最大消费金额的上四分位数作为阈值，如果客户的最大消费金额有大于该阈值，则将该客户定义为有高端消费，反之则无高端消费。

>实训任务
>+ 请根据`user_features`表中最大消费金额列(`max_consume_amt`)的上四分位数定义高端消费的阈值，命名为`threshold`。
>+ 请根据阈值，判定客户是否拥有高端消费，'是'则标记为1 、'否'则标记为0，并将结果存储在`user_features`表的`high_consumption`列中列中。
>  + 提示：使用`quantile()`可计算四分位数。上四分位数等于所有数值由小到大排列后，75%位置上的数字。

In [ ]:
import pandas as pd

# 计算阈值
threshold = user_features['max_consume_amt'].quantile(0.75)

# 判定有无高端消费
user_features["high_consumption"] = user_features['max_consume_amt'].apply( lambda x : 1 if x > threshold else 0)

print(user_features.head())

## 5.6 规则类标签的计算（是否休眠客户）
精准营销的另一个目标是激活“休眠”客户。所谓“休眠”客户，是指那些已经了解企业和产品，却还在消费与不消费之间徘徊的客户。

“休眠”客户对于企业的发展具有重大作用，调查显示“休眠”客户的消费能力是普通客户消费能力的3〜5倍，甚至更高，而企业挖掘新客户的花费是“休眠”客户的8倍，所以唤醒休眠客户对企业开展精准营销具有重大意义。

取出每个客户的交易记录，设定交易次数的下四分位数为阈值，交易次数`total_transactions_cnt`小于阈值的客户则视为休眠客户 (1) ,交易次数大于等于阈值的客户则视为活跃客户 (0)。

>实训任务
>+ 请根据`user_features`表中交易次数列(`total_transactions_cnt`)的下四分位数定义是否休眠的阈值，命名为`threshold`。
>+ 请根据阈值，判定客户是否为休眠客户，'是'则标记为1 、'否'则标记为0，将结果保存在`user_features`表的`sleep_customers`列中。
>  + 提示：使用`quantile()`可计算四分位数。下四分位数等于所有数值由小到大排列，25%位置上的数字。

In [ ]:
import pandas as pd

# 计算阈值
threshold = user_features['total_transactions_cnt'].quantile(0.25)

# 判定是否休眠
user_features['sleep_customers'] = user_features['total_transactions_cnt'].apply(lambda x : 1 if x < threshold else 0)

print(user_features.head())

## 5.7 规则类标签的计算（RFM模型介绍）
+ 对于每一次消费来说，不同的客户都有不同的消费时间，并且这种消费时间呈现出一定的规律性，整理这方面的标签有助于全方位了解客户，并决定在哪个时间段开展营销活动。

+ 消费频率和消费金额是了解产品是否受欢迎的主要考量依据，也是区分客户的重要标准。所以我们要详细记录客户在这两个方面的信息，并对此进行分析。

在众多客户关系管理模型（CRM）中，RFM被广泛使用。RFM模型是衡量客户价值和客户创利能力的重要工具和手段。该模型通过一个客户的购买时间、购买总体频率以及购买金额三项指标来描述该客户的价值状况。

RFM模型较为动态地层示了一个客户的全部轮廓，这对个性化的沟通和服务提供了依据，同时，如果与该客户打交道的时间足够长，也能够较为精确地判断该客户的长期价值(甚至是终身价值)，通过改善三项指标的状况，从而为更多的营销决策提供支持。

RFM模型三个要素，分别是：Recency、Frequency、Monetary。

+ Recency（近度）：最近一次消费距离观察点的天数，上一次消费时间越近的客户应该是比较好的客户，对提供即时的商品或是服务最有可能响应。

+ Frequency（频度）：客户在一段时间内消费的次数，通常来说最常消费的客户，忠诚度相对高于其它客户。

+ Monetary（值度） ：客户在一段时间内消费的总金额，消费金额的意义不言而喻。

<center><img src=".\Pics\Pic5_4.jpg" width="500" height="300" alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图5.4：RFM模型</font></center>

## 5.8 规则类标签的计算（计算时间差）
近度是维系客户的一个重要指标。最近购买商品或是使用平台进行支付的消费者，是最有可能再使用这个平台的客户。再则，要吸引一个几个月前才上门的客户，比吸引一个一年多以前来过的客户要容易得多。营销人员如接受这种强有力的营销哲学——与客户建立长期的关系而不仅是卖东西，会让顾客持续保持往来，并赢得他们的忠诚度。

接下来我们就要准备计算每个客户的近度`recency`，即距离观察点的时间差。在前面的步骤中，我们依据交易频度随时间的变化，限定了观察的有效时间，即2016年7月1日到2017年12月31日。所以在RFM模型中，我们选取的观察点为2018年1月1日，由此就可以根据每笔交易的时间计算距离天数。RFM模型的前提是提取只属于消费类的数据，在5.4步骤中已计算过，计算步骤为从`data`中选取金额流出的交易记录，然后再剔除转账、体现、转入、还款等记录，我们在这里回顾一下`consume`表。

<center><font size="2.5px" face="微软雅黑" color="666666">表5.6：consume表内容示例</font></center>
<center><img src=".\Pics\Table5_6.jpg" width="750" height="250" alt="cursor" align=center /></center>

>实训任务
>+ `consume`表在题目中已预先读入，保存在DataFrame对象 `consume`中。
>+ 选取观察点为`2018-01-01`，保存在变量`standard_day`中，此步骤题目已默认给出。
>+ 请新建一列`distance`，计算`consume`中每次交易距离观察点的天数，保存在`consume['distance']`中。
>+ 提示:
>  + 使用观察点的时间(`standard_day`)减去交易时间(`pay_time`)，再除以一天的时间间隔，即可计算距离的天数。
>  + 在Pandas中`Timedelta`表示两个`datetime64`对象之间的时间差，如`pd.Timedelta(days=1)`代表一天的时间差。

In [ ]:
import pandas as pd
import numpy as np

# 选取观察点
standard_day = pd.Timestamp(2018,1,1)

# 计算天数
consume['distance'] = (standard_day - consume['pay_time']) / pd.Timedelta(days=1)

print(consume.head())

## 5.9 规则类标签的计算（近度、频度、值度）
在上一步，我们设定了日期的观察点，并计算了每笔交易距离观察点的距离天数。现在我们需要将交易记录根据客户进行分组，计算每个客户的RFM值。客户RFM值的具体计算方法为：

+ `recency` (近度) ：将交易记录根据客户进行分组后，求出每个客户的交易距离最小值。
+ `frequency` (频度) ：将交易记录根据客户进行分组后，求出每个客户的消费笔数。
+ `monetary` (值度) ：将交易记录根据客户进行分组后，求出每个客户的消费总额。

最终的`rfm`三列结果示例如下：

<center><font size="2.5px" face="微软雅黑" color="666666">表5.7：rfm三列示例</font></center>

 |user_id	|recency	|frequency	|monetary|
|:-:|:-:|:-:|:-:|
|109464|	85.212512|	226|	44662.59|
|115043|	41.000000	|367|	92114.84|
|125322	|35.000000	|344|	670721.81|
|131673	|75.000000	|67	|18161.48|
|136544	|75.000000	|158	|44836.38|

>实训任务
>+ 题目已预先读入上一步的结果 `consume`。
>+ 请根据RFM的计算方法，对`consume`表按客户进行分组，通过聚合函数计算每个客户的近度、频度和值度，保存在`user_features`表的`recency`、`frequency`和`monetary`三列中。

In [ ]:
import pandas as pd

# 计算rfm
user_features[['recency','frequency','monetary']] = consume.groupby('user_id').agg({'distance':'min', 'user_id':'size', 'payment':'sum'})

print(user_features.head())

## 5.10 规则类标签的计算（RFM总得分）
上一步我们已经通过分组和聚合运算得到了每个客户RFM的具体值，根据这三个指标，我们又把客户分别进行四等分，得到不同指标的评分阶梯，根据阶梯计算客户RFM总得分。客户RFM总得分的计算方法如下：
+ 将最近一次消费按时间距离进行降序排序，前25％标记为4，25％－50％标记为3，50％－75％标记为2，75％－100％标记为1。
+ 将消费频次及消费金额按数字大小进行升序排列，前25％标记为1，25％－50％标记为2，50％－75％标记为3，75％－100％标记为4。
+ 将每个客户对应的三个评分标记相加，作为客户RFM的总得分(`Total_Score`)。

<center><font size="2.5px" face="微软雅黑" color="666666">表5.8：客户RFM评级</font></center>

|RFM总得分|客户RFM评级|
|:-:|:-:|
|9-12分|优质客户|
|5-8分|一般客户|
|1-4分|低贡献客户|

理论上来说，同等的资源投入的情况下，一名优质客户带来的回报将会是一般客户的5倍，可以推出，在资源有限的前提下，满足客户的顺序应该也是自上而下的。

>实训任务
>+ 请将`user_features`表中的`recency`、`frequency`、`monetary`三列进行等频离散化，分成4个区间，区间命名为得分1、2、3、4。将结果分别保存在`user_features`表的`R_score`、`F_score`、`M_score`三列中，其中`recency`越近得分越高，`frequency`、`monetary`越大得分越高。
>+ 前面计算的近度、频度和值度标签中，有些客户可能没有消费记录而导致该三列为空，我们需要将这些空值填为1(最低得分)，此步骤题目已默认给出。
>+ 请计算RFM总得分：对`R_score`、`F_score`、`M_score`三列的评分进行数值加和，保存在`user_features`表的`Total_Score`列中，此步骤题目已默认给出。

+ 结合`R_score`、`F_score`和`M_score`这三个评分，我们就可以把客户分成`4＊4＊4 = 64`类。
+ 对三个评分指标进行加和，根据总得分`Total_score`又可以把客户分成4类。对每类客户进行分析，然后制定营销策略。

In [ ]:
import pandas as pd

# 等频离散化
user_features['R_score'] = pd.qcut(user_features['recency'], 4, labels=[4, 3, 2, 1])
user_features['F_score'] = pd.qcut(user_features['frequency'], 4, labels=[1, 2, 3, 4])
user_features['M_score'] = pd.qcut(user_features['monetary'], 4, labels=[1, 2, 3, 4])

# 填充缺失值
user_features[['R_score','F_score','M_score']] = user_features[['R_score','F_score','M_score']].fillna(1)

# 计算总得分
user_features['Total_Score'] = user_features.R_score.astype('int') + user_features.F_score.astype('int') + user_features.M_score.astype('int')

print(user_features.head())

## 5.11 RFM可视化分析
前面的步骤中，我们已经计算好了`recency`、`frequency`、`monetary`以及其对应的得分，并得到了单个客户的RFM总得分`Total_Score`，得分取值范围为3-12。现在我们可以将它们进行一些组合，可视化分析其中的一些规律。如将每个`Total_Score`的取值作为一个类别进行分组，统计不同`Total_Score`下RFM的取值分布情况。

> 实训任务
> + 请使用`groupby()`函数对`user_features`根据`Total_Score`列进行分组，分别计算每组中`recency`、`frequency`、`monetary`的平均值。接着使用Pandas绘图机制，调用`plot()`函数绘制三个柱形图，分别放置在子图`ax1`,`ax2`和`ax3`中。
>   + 要求：设置三个子图的颜色图参数`colormap`为`Blues_r`，并将x轴刻度旋转360度，三个子图的`y`轴标签分别为`recency`、`frequency`、`monetary`。画布和子图对象的创建已默认给出。

从图中可以看到：
+ 随着`Total_Score`的增大，`recency`的平均值逐渐减小，这也印证了较优质的客户群体，最近一次消费也普遍较近；
+ 随着`Total_Score`的增大，`frequency`、`monetary`的平均值逐渐增大，这也印证了越优质的客户群体，消费频率和消费金额普遍越高；

此外消费金额也可以验证“帕雷托法则”(Pareto’s Law)——80％的消费来自20％的顾客。从图中可看出排名前20％的顾客所花费的金额占平台所有消费额的60％-80%。

至此，项目中的9个规则类标签已全部提取。接下来我们需要提取预测类标签，即通过建立分类或回归模型去预测客户的某些属性。

In [ ]:
import matplotlib.pyplot as plt

fig,[ax1,ax2,ax3] = plt.subplots(1,3,figsize=(16,4))

# 绘制条形图
user_features.groupby('Total_Score')['recency'].mean().plot(kind='bar', colormap='Blues_r', rot=360, ax = ax1)

user_features.groupby('Total_Score')['frequency'].mean().plot(kind='bar', colormap='Blues_r', rot=360, ax = ax2)

user_features.groupby('Total_Score')['monetary'].mean().plot(kind='bar', colormap='Blues_r', rot=360, ax = ax3)


# 设置y轴标签
ax1.set_ylabel('recency')
ax2.set_ylabel('frequency')
ax3.set_ylabel('monetary')

# 自动调整子图间距
plt.tight_layout()
plt.show()

## 5.12 客户价值等级数据的读取
在建立了事实类标签和规则类标签之后，我们对其进行了合并保存在了DataFrame变量`user_features`中，现在共有49个标签。此外本项目还包含另外一部分数据，数据中包含客户的价值等级，数据表名为`user_potential`，字段如下：

<center><font size="2.5px" face="微软雅黑" color="666666">表5.9：客户价值等级字段说明</font></center>

|英文名称|中文名称|备注|
|:---:|:---:|:---:|
|user_id|客户id|每个客户对应一个id |
|user_potential|客户价值等级|取值为0/1，分别代表低/高价值客户 |


但是数据中的客户价值等级不太完整，只有4000名左右的客户有价值等级数据，3600名左右的客户没有价值等级。我们根据提取出的事实类和规则类标签建立模型，对没有客户价值等级标签的客户进行预测。需要借助模型进行预测的标签我们称之为预测类标签，在这个项目中，客户价值等级为预测类标签。接下来我们从数据库中读取客户价值等级数据。

> 实训任务
> + 请补充完成创建数据库连接对象`connection`的语句。
> + 请书写`SQL`语句并赋予变量`sql`，从数据库中查询表`user_potential`的所有行，使用`cursor`获取结果，并存放于变量`level`中
> + 请将`level`转换成DataFrame格式，列名设置为`user_id`,`user_potential`。

提示：`connection`对象中的参数为：
 + `host`：MySQL服务器地址。
 + `port`：连接的端口号，MySQL数据库默认端口是`3306`。
 + `user`和`password`：登陆数据库的用户名和密码。
 + `db`：要连接的数据库名字。
 + `charset`：字符编码，我们设置为`utf8mb4`，因为它可以很好地支持中文。

表5.10显示了创建连接对象`connection`所需要的参数信息：

<center><font size="2.5px" face="微软雅黑" color="666666">表5.10：MySQL连接参数表</font></center>

|参数名称|参数信息|参数类型|
|:----|:-----|:----|
|host|traindb.cookdata.cn|string|
|port|3306|int|
|user|raa_user|string|
|password|bigdata123|string|
|db|transactions|string|
|charset|utf8mb4|string|

In [ ]:
import pandas as pd
import pymysql

# 从数据库读入客户价值等级数据
connection = pymysql.connect(host='traindb.cookdata.cn', 
                           port = 3306, 
                           user = 'raa_user', 
                           password = 'bigdata123',
                           db = 'transactions',
                           charset = 'utf8mb4')

try:
    with connection.cursor() as cursor:
        sql = "SELECT * FROM user_potential"
        cursor.execute(sql)
        level = cursor.fetchall() 
finally:
    connection.close()

level=pd.DataFrame(list(level), columns=('user_id', 'user_potential'))

print(level.head())

## 5.13 One-Hot编码
在建立模型之前我们需要对已经提取的标签进行处理，对于有缺失的标签，我们需要进行填充。消费渠道`consumption_channel`列为离散型变量，我们统一将空值填充为`OthersPay`，代表其他消费渠道。其他为空值的列均为连续性变量，我们将这些空值填为0，代表客户不具有这部分的交易行为。缺失值的填充步骤为题目预先完成，接下来我们还要对标签做一些其他的处理，如字符型标签进行编码、连续性标签进行离散化等。标签处理完成后进行训练测试集划分，接着训练模型并通过模型预测客户价值等级。

以上所做的操作只是为了更好的建立模型来预测客户价值等级，原始客户标签表`user_features`中的数据应当保留，不进行任何的数据处理。客户等级标签预测完成后，将其合并作为`user_features`表的最后一列。

为了保持`user_features`中标签数据不变，我们在这里复制一份`user_features`，保存为表`user_features_predict`，在建模过程中的操作均在`user_features_predict`表中。

<center><img src=".\Pics\Pic5_5.jpg"width=700 height=200 alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图5.5：客户价值等级预测流程</font></center>


消费渠道标签统计客户在哪个平台拥有最多的交易记录，取值包括`AliPay`，`WePay`，`UnionPay`，`CreditCardPay`和`OthersPay`。针对这一字符型标签，我们需要进行数值编码，方便模型进行运算。One-Hot编码是将包含K个取值的字符型标签转换成K个取值为0或1的二元标签。

我们通过One-Hot编码将消费渠道`consumption_channel`转换为五个二元标签，分别为：支付宝(`AliPay`)，微信(`WePay`)，银联(`UnionPay`)，信用卡(`CreditCardPay`)，其他(`OthersPay`)。	

Pandas中的`get_dummies()`函数对数据进行One-Hot编码，语法格式为`pandas.get_dummies(data, prefix=None, prefix_sep='_',columns=None)`
+ `data`：需要处理的数据。
+ `prefix`：设置新列名名字的前缀，默认为原标签列名。
+ `prefix_sep`：前缀和原属性名之间的间隔，默认为`_`。
+ `columns`：选取数据中的哪些列需要One-Hot编码，默认为数据中所有`object`类型。

对于`prefix`和`prefix_sep`的理解，我们举一个例子。如性别列(列名`sex`)有两种属性(`male`和`female`)，要将其进行One-Hot编码。如果不设置`prefix`和`prefix_sep`，则新列名为`sex_male`和`sex_female`。如果设置`prefix='性别'，prefix_sep='~'`则新列名为`性别~male`和`性别~female`。

> 实训任务
> + 请对`user_features_predict`表中的消费渠道`consumption_channel`列进行One-Hot编码。
>   + 要求：不保留原始标签，只保留One-Hot后的新标签，且新标签的列名分别为`AliPay`、`WePay`、`UnionPay`、`CreditCardPay`、`OthersPay`。

可以看到，原始的列经过One-Hot编码转换成了对应新的列。

In [ ]:
import pandas as pd

# 对消费渠道进行One-hot编码
user_features_predict = pd.get_dummies(
    data = user_features_predict.consumption_channel,
    prefix = None,
    prefix_sep = None,
    columns = ['AliPay', 'WePay', 'UnionPay', 'CreditCardPay', 'OthersPay'],
)

print(user_features_predict.head())

## 5.14 日期等距离散化
在处理连续型数据时，为了便于分析需要将其进行离散化，即把数据放入一个个小区间中。之前提取的标签中有两个时间类标签，分别是网购首单时间`online_buy_first_date`和最近一次网购时间`online_buy_last_date`，在这里我们对其进行等距离散化，即按照相同的时间跨度将数据划成几等份。

可以使用Pandas中的`cut()`函数进行划分，语法如下：`pandas.cut(x，bins，labels = None，duplicates =raise)`。
+ `x`：1维ndarray或Series，表示要划分的数据。
+ `bins`：为整数，即将`x`划分为多少个等间距的区间。
+ `labels`：设置区间名称，可以为一个列表或数组。若`labels`为`False`则返回整数填充的区间名。
+ `duplicates`：如果分组的`bin`边缘不唯一，`duplicates`为`raise`则报告`ValueError`，为`drop`则直接删除非唯一值。

> 实训任务
> + 请将`user_features_predict`表中的网购首单时间标签进行等距离散化，区间数设为`6`，设置`labels`为`False`，且删除`bin`边缘的非唯一值，结果保存在原列中。
> + 请将`user_features_predict`表中的网购最后一次时间标签进行等距离散化，区间数设为`6`，设置`labels`为`False`，且删除`bin`边缘的非唯一值，结果保存在原列中。

可以看到，经过离散化之后，两个标签中的值已经划分在了6个区间中，并命名为了相应了数字。

In [ ]:
import pandas as pd

# 对网购首单时间进行离散化
user_features_predict['online_buy_first_date'] = pd.cut(
    x = user_features_predict.online_buy_first_date,
    bins = 6,
    labels = False,
    duplicates = "drop",
)

# 对网购最后一次时间进行离散化
user_features_predict['online_buy_last_date'] = pd.cut(
    x = user_features_predict.online_buy_last_date,
    bins = 6,
    labels = False,
    duplicates = "drop",
)

print(user_features_predict[['online_buy_first_date','online_buy_last_date']])

## 5.15 连续型数值等频离散化
对数据进行离散化时，若每个区间的间隔是相等的，称作等距离散化。另外一种叫做等频离散化，它是指每个区间的样本数相同。我们可以发现对于该数据集，连续型数值多呈长尾分布，若用等距离散化会在分布较少的区域会分出很多无意义的区间，在这里我们对连续型数值进行等频离散化。

Pandas提供了`qcut()`函数可实现等频离散化，自动分配每个面元的区间，其函数语法为：`qcut(x, q, duplicates='raise')`，其中：
+ `x`：1维ndarray或Series，表示要划分的数据。
+ `labels`：设置区间名称，可以为一个列表或数组。若`labels`为`False`则返回整数填充的区间名。
+ `q` ：分位数，用来指定区间的数量，表示要划分为几组。
+ `duplicates`： 如果分组的`bin`边缘不唯一，`duplicates`为`raise`则报告`ValueError`，为`drop`则直接删除非唯一值。

> 实训任务
> + 请对`user_features_predict`中取值为连续型数值的列`continuous_col`进行等频离散化。
>   + 要求：区间数设为6，划分后的区间名称`labels`设为`False`，且删除`bin`边缘的非唯一值，最终结果直接保存在原始的列中。

可以看到，进行等频离散化后，所有的连续数值都放置到了各自的区间中。

In [ ]:
import pandas as pd

# 所有数值为连续型的列
continuous_col = ['max_consume_amt', 'consume_order_ratio', 'mon_consume_frq', 'online_cnt', 'online_amt', 'online_avg_amt', 'mon_online_frq','dining_cnt','dining_amt', 'dining_avg_amt', 'business_travel_cnt','business_travel_amt', 'business_travel_avg_amt', 'mon_business_travel_frq', 'car_cnt', 'car_amt', 'phone_fee_cnt', 'phone_fee_amt', 'credit_card_repay_cnt', 'credit_card_repay_amt','cash_advance_cnt', 'cash_advance_amt', 'payroll','total_transactions_amt', 'total_transactions_cnt', 'withdraw_cnt','withdraw_amt', 'total_deposit', 'total_withdraw', 'transfer_cnt','transfer_amt', 'transfer_mean', 'internet_media_cnt', 'internet_media_amt', 'public_pay_amt', 'return_cnt',  'recency', 'frequency', 'monetary']

# 对所有连续型数值的列进行等频离散化
user_features_predict[continuous_col] = user_features_predict[continuous_col].apply(lambda x : pd.qcut(
        x,
        labels = False,
        q = 6,
        duplicates = "drop"
))
    
print(user_features_predict.head())

## 5.16 训练测试集的划分
在前面已经从数据库中读取客户价值等级数据，我们根据`user_id`将客户的价值等级(`user_potential`)作为一列合并到`user_features_predict`表中，此时`user_features_predict`表的结构如表5.12所示。

<center><font size="2.5px" face="微软雅黑" color="666666">表5.12：user_features_predict表内容示例</font></center>
<center><img src=".\Pics\Table5_12.jpg" width="750" height="250" alt="cursor" align=center /></center>

原始的客户价值等级标签并不完整，存在缺失值。为了解决这个问题，我们将带标签的完整数据训练模型，通过训练完成的模型来预测存在缺失值的客户价值等级标签。
我们需要从`user_features_predict`表中筛选出`user_potential`列不为空的客户数据，保存在变量`customers`中，输送给模型使用。在建立客户价值等级预测模型之前，首先需要划分训练集和测试集，将在训练集上训练好的模型放在测试集进行检验，以验证模型的准确性。我们将客户的所有事实类标签和规则类标签均作为特征进行训练。

在Sklearn中的`model_selection`模块，存在`train_test_split()`函数，用作训练集和测试集划分，函数语法为：`train_test_split(x,y,test_size = None,random_state = None)`，其中：
+ `X,y`: 分别为预测所需的所有标签，以及需要预测的标签(即客户价值等级)。
+ `test_size`: 测试集比例，例如`test_size=0.2`则表示划分`20%`的数据作为测试集。
+ `random_state`: 随机种子，因为划分过程是随机的，为了进行可重复的训练，需要固定一个`random_state`，结果重现。
+ 函数最终将返回四个变量，分别为`X`的测试集和训练集，以及`y`的测试集和训练集。

> 实训任务
> + 数据已经进行了分离为`X`, `y`，分别为预测所需的所有标签，以及需要预测的标签(即客户价值等级)。
> + 请使用`train_test_split()`函数进行训练集和测试集的划分，测试集比例为`0.2`，随机种子`random_state`设定为`33`。
> + 请将返回的变量命名为`x_train`, `x_test`, `y_train`, `y_test`，类型为ndarray数组对象。

从`user_features_predict`表中提取列有价值等级的客户数据，保存在变量`customers`中。经过划分，我们已经将`customers`中的数据划分为训练集和测试集，接下来我们将训练集带入算法中进行模型训练。

In [ ]:
import pandas as pd
import numpy as np

# 选取观察点
standard_day = pd.Timestamp(2018,1,1)

# 计算天数
consume['distance'] = (standard_day - consume['pay_time']) / pd.Timedelta(days=1)

print(consume.head())

## 5.17 构建客户价值等级预测模型
前面已经将有价值等级的客户数据进行了训练集和测试集的划分，现在我们需要选择一个分类模型进行训练，再用此模型去预测价值等级未知的客户。分类算法有很多种，我们选择尝试一个较为简单的模型——逻辑回归。

#### 逻辑回归简介：
我们知道，在线性回归中预测目标$y$是连续型，且可以写成样本$\boldsymbol{x}$每一个特征的线性加权形式：
\begin{equation}
y = w_1x_1 + \cdots + w_dx_d + w_0= \boldsymbol{w}^{\rm T} \boldsymbol{x}
\end{equation}

其中$\boldsymbol{w}$为参数向量，$d$为特征维度。
现在，假设我们解决的是二分类问题。为了能够利用回归的方法来解决二分类问题，我们需要对线性回归的输出作改变。
逻辑回归采用的方法是引入一个Logistic函数，将连续型的输出映射到$(0,1)$之间。
Logistic函数的定义如下：
\begin{equation}
\sigma(x)=\frac{1}{1+e^{-x}}
\end{equation}
对应的函数曲线如下图所示，
当输入$x$很大或很小时，该函数以接近于0或1的值输出，且$\sigma(0)=0.5$。

<center><img src=".\Pics\Pic5_6.png" width="400" alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图5.6：Logistic函数</font></center>

有了Logistic函数$\sigma(x)$，我们可以将任意实数映射到$(0,1)$之间。
此时样本属于正类($y=1$)的概率可以表示为：
\begin{equation}
p(y=1 |\boldsymbol{x}_i) = \frac{1}{ 1 + e^{-\boldsymbol{w}^{\rm T} \boldsymbol{x}_i}}
\end{equation}
逻辑回归在线性回归的基础上，利用一个非线性函数，建立了二元预测目标与原始输入之间的关系。

下面我们通过逻辑回归根据已有的客户价值等级标签对无标签的客户进行预测。

在sklearn的`linear_model`模块中，可以直接调用`LogisticRegression()`函数创建模型。再将划分好的训练集`x_trian`，`y_train`带入模型中进行训练。

模型训练好后便可带入测试集`x_test`进行预测。预测通常有两种方式：`predict()`和`predict_proba()`，前者直接预测出分类标签（0或1），后者则预测出目标分数为正类(1，代表高价值客户)或负类(0，代表第价值客户)的概率估计(取值在0-1之间)。

对于二分类问题常用的评价指标有准确率(`Accuracy`)、精度(`Precision`)、召回率(`Recall`)和F1值(`F1 Score`)。通常以关注的类(高价值等级)为正类，其他类(低价值等级)为负类。四个评价指标的计算方法如下：
+ $Accuracy = \frac{所有正确预测的样本数}{总样本数}$
+ $Precision = \frac{预测为正类且正确预测的样本数}{所有预测为正类的样本数}$
+ $Recall = \frac{预测为正类且正确预测的样本数}{所有真实情况为正类的样本数}$
+ $F_1 Score = 精确率和召回率的调和平均数 = 2 \frac{Precision \times Recall}{(Precision+Recall)}$

> 实训任务
> + 请使用`LogisticRegression()`新建一个模型对象，命名为`clf`。
> + 请对`clf`对象调用`fit`方法，对训练集(`x_train`， `y_train`)进行训练。
> + 请对训练好的`clf`模型调用`predict()`函数，对测试集(`x_test`)进行预测，将结果保存到变量(`y_predict`)中。
> + 请对训练好的`clf`模型调用`score()`函数计算模型对测试集分类的准确率，再使用`classification_report()`函数输出分类报告。此部分题目已默认给出。

通过以上步骤，我们已经训练好了一个简单的逻辑回归模型，并在测试集上做了相应的预测，可以看到模型预测的正确率为72%。接下来我们使用这个模型对价值等级标签缺失的客户进行预测。

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 设置分类器
clf = LogisticRegression()

# 训练
clf.fit(x_train, y_train)

# 预测
y_predict = clf.predict(x_test)

# 计算准确率
score = clf.score(x_test,y_test)

# 以百分数形式输出并保留两位小数
print('%.2f%%'%(score*100))

# 输出分类报告
report = classification_report(y_test,y_predict)
print(report)

## 5.18 预测客户价值等级
上一步我们建立了逻辑回归模型，并在测试集上进行了检验，达到了72%的准确率。下面我们用建立好的模型对价值等级未知的客户进行预测，并将预测完成的客户数据与价值等级已知的客户数据进行拼接，与原始`user_features`表进行合并。

我们首先从`user_features_predict`表中选取价值等级未知的客户数据，保存在变量`null_customers`中，带入构建好的模型进行预测。

> 实训任务
> + 请从`user_features_predict`表中选取客户价值等级(`user_potential`)为空的客户数据，保存在DataFrame对象`null_customers`中。
> + 使用上一步已构建好的模型`clf`，带入`null_customers`中除去客户价值等级列的数据进行预测，将预测结果保存在`null_customers["user_potential"]`列中。
> + 请将客户价值等级已知和预测完成的两部分客户价值等级标签进行拼接，根据`user_id`合并在原始客户标签表`user_features`的`user_potential`列中。

可以看到，我们已经对价值等级未知的客户进行了预测，并统一合并到`user_features`表中。

In [ ]:
import pandas as pd

# 选取价值等级未知的客户数据
null_customers = user_features_predict[user_features_predict['user_potential'].isnull()]

# 对价值等级未知的客户进行预测
null_customers["user_potential"] =  clf.predict(null_customers.drop(['user_potential'], axis = 1))

# 将数据合并在原始表中
user_features['user_potential'] = pd.concat([customers['user_potential'],null_customers['user_potential']],axis=0)

print(user_features["user_potential"].value_counts())

## 5.19 客户价值等级分布
上一步我们对客户价值等级标签做了预测，并将数据整体合并保存在了`user_features`表中。下边我们通过可视化的方式来观察一下客户价值等级的数量分布。

Pandas中的`plot()`函数可以绘制各种图形，设置参数`kind='bar'`可以绘制柱状图。下面我们利用`plot()`函数绘制不同客户价值等级的数量分布柱状图。

> 实训任务
> + 请计算不同价值等级的客户数量，即对`user_features`表中的客户价值等级列使用`value_counts()`进行计数，参数默认，保存在变量`lmt_num`中。
> + 请调用Pandas中的`plot()`函数绘制柱状图，查看客户价值等级的取值分布情况。设置`x`轴刻度标签旋转`360`度，柱形宽度`width`设为`0.15`。
> + 请设置柱形名称分别为`低价值`,`高价值`。默认的名称为标签值`0`、`1`，使用`plt.xticks()`函数可更改柱形名称。
> + 请设置`x`轴标签为`客户价值等级`，`y`轴标签为`客户数量`。
> + 请设置标题为`客户价值等级分布图`。

从图中我们可以看到，在所有客户中，低价值等级客户的人数较多，大约是高价值等级客户的两倍。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 计算不同客户价值等级的客户数量
lmt_num = user_features["user_potential"].value_counts()

fig = plt.figure(figsize=(8,6))

# 绘制柱状图，查看客户价值等级取值分布情况
lmt_num.plot(kind='bar', rot=360, width=0.15)

# 设置柱形名称
plt.xticks([0,1], ['低价值', '高价值'])

# 设置x、y轴标签
plt.ylabel('客户数量')
plt.xlabel('客户价值等级')

# 设置标题
plt.title('客户价值等级分布图')

plt.show()

## 5.20 客户标签间的相关性
对客户价值等级进行预测后，共有50个客户标签，现在我们分析一下各标签之间的关联程度，使用皮尔森相关系数来计算不同标签之间的相关性。

皮尔森相关系数是用来反映两个变量线性相关程度的统计量，是一种线性相关系数，一般用r表示。r的取值在-1与+1之间。
+ 若r&gt;0，表明两个变量是正相关，即一个变量的值越大，另一个变量的值也会越大。
+ 若r&lt;0，表明两个变量是负相关，即一个变量的值越大另一个变量的值反而会越小。
+ r的绝对值越大表明相关性越强。

将各标签间的相关系数通过绘制热力图的方式，可以非常方便的进行整体观察。热力图是数据可视化中比较常用的显示方式，颜色的深浅表示了描述量的大小，如图5.5所示。

<center><img src=".\Pics\Pic5_7.png" width="550" height="125" alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图5.7：热力图示例</font></center>

在标签数量较多的情况下，很难直接观察到各个标签之间的相关性，这里我们选取后15列标签，计算它们之间的相关性并用热力图表示。

我们利用Seaborn中的`heatmap()`函数来绘制热力图。`heatmap()`函数语法为：
`seaborn.heatmap(data,annot=None,linewidths=0,cmap=None)`。其中：
+ `data`：数据，可以是NumPy数组ndarray，也可以是Pandas的Dataframe。
+ `annot`：`annot`默认为`False`，当`annot`为`True`时，在`heatmap`中每个方格写入数据。
+ `linewidths`：设置热力图矩阵之间的间隔大小。
+ `cmap`：设置颜色风格。

> 实训任务
> + 请取出`user_features`中的后15列，保存在变量DataFrame变量`last_features`中，此步骤已默认给出。
> + 请用`corr()`函数计算`last_features`各标签间的皮尔森相关系数，保存在变量`corr`中，这里相关性取绝对值。
>   + 注意：计算标签之间的相关性时，默认计算数值型的列。项目中的`R_score`、`F_score`和`M_score`三列为离散化之后的`category`型变量，不计算其相关性。
> + 请调用Seaborn中的`heatmap()`函数绘制热力图，设置参数`annot=True`，代表在每个方格中写入数据，并设置参数`linewidths = 0.05`，代表将方格的间隔设置为`0.05`，参数`cmap`已事先规定。

通过观察热力图的我们可以发现`Total_score`、是否拥有高端消费等与客户价值等级的相关性相对较高。

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 取所有标签的后15列
last_features = user_features.iloc[:,-15:]

# 计算各标签间的皮尔森相关系数
corr = last_features.corr().abs()

fig = plt.figure(figsize = (10,10))

# 设置颜色风格
cmap = sns.cubehelix_palette(8, start = 2, rot = 0, dark = 0, light =0.95)

# 绘制热力图
sns.heatmap(data = corr, annot=True, linewidths=0.05, cmap=cmap)

plt.show()

## 5.21 客户价值等级与是否拥有高端消费的可视化分析
在上一步中我们发现是否拥有高端消费(`high_consumption`)与客户价值等级(`user_potential`)之间的相关性较高。下边我们就来观察一下在有高端消费和无高端消费的人群中，不同客户价值等级的人数分布。

对于这种多重分组计算个数的问题，我们通常使用交叉表来计算。交叉表是一种常用的分类汇总表格，用于频数分布统计。Pandas中的`crosstab()`函数可以计算交叉表，语法如下：
`pandas.crosstab(index, columns,  rownames=None, colnames=None)`
+ `index`：要在行上进行分组的值。
+ `columns`：要在列上进行分组的值。

最终对两个特征进行交叉分组，再分别对索引和列名进行重命名，结果如下：

<center><font size="2.5px" face="微软雅黑" color="666666">表5.12：交叉表内容示例</font></center>

|  分组特征  |低价值客户  |  高价值客户|
|:-:|:-:|:-:|
|无高端消费   |   4470   |   1275|
|有高端消费  |      779   |   1106|

## 实训任务
> + 请使用交叉表计算`user_features`表中是否拥有高端消费和客户价值等级之间的分组频数分布，保存在DataFrame对象`counts`中。
> + 请对`counts`重命名索引和列名，分别为`"无高端消费","有高端消费"`，` "低价值客户","高价值客户"`。
> + 请调用Pandas中的`plot()`函数绘制柱状图，查看客户价值等级与是否拥有高端消费的关系，要求设置`x`轴刻度标签旋转`360`度，柱形宽度`width`设为0.3。
> + 请设置`x`轴标签为`是否拥有高端消费`，`y`轴标签为`人数`。
> + 请设置标题为`客户价值等级与是否拥有高端消费之间的关系`。

由图，清晰可见在无高端消费的人群中，高价值等级客户的占比较小；在有高端消费的人群中，高价值等级客户的占比更多；说明客户价值等级高的群体更倾向买贵的物品。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 计算交叉表
counts = pd.crosstab(index=user_features['high_consumption'], columns=user_features['user_potential'])

# 重命名索引和列名
counts.index = ['无高端消费', '有高端消费']
counts.columns= ['低价值客户', '高价值客户']

fig = plt.figure(figsize=(8,6))

# 绘制柱状图
counts.plot(kind='bar', rot=360, width=0.3)

# 设置x、y轴标签
plt.xlabel('是否拥有高端消费')
plt.ylabel('人群')

# 设置标题
plt.title('客户价值等级与是否拥有高端消费之间的关系')

plt.show()

## 5.22 月均消费频度的可视化分析
在前面我们将客户价值等级与是否拥有高端消费进行组合，现在我们将月均消费频度(`mon_consume_frq`)和客户价值等级(`user_potential`)这两个特征进行组合分析。月均消费频度是一个连续型的标签，我们需要先将其离散化成不同的区间，再通过分组的方式探索不同区间中客户价值等级之间的关系。我们首先观察月均消费频度的取值分布，画出月均消费频度的取值分布直方图以及核密度估计曲线。

Seaborn中的`distplot()`函数可以绘制直方图，直方图首先将数据等距地分为几组，然后每一组绘制一个长方形条，长方形条的高度表示落在本组内样本的数量（频数分布直方图）或是占比（频率分布直方图）， `distplot()`函数的主要参数及说明如下：
+ `a`： 需要进行绘图的特征，通常为Series对象。
+ `color` ： 设置条形的颜色。
+ `bins` ：设置将数据分为几组。
+ `kde` ：是否添加核密度估计曲线，默认为`True`。

下面我们利用Seaborn中的`distplot()`函数绘制月均消费频度的取值分布直方图。

> 实训任务
> + 请绘制`user_features`表中月均消费频度(`mon_consume_frq`)的取值分布直方图，颜色设置为`pink`，并在图上添加核密度估计曲线。
> + 请将标题设置为`月均消费频度取值分布`。
> + 请设置`x`轴标签为`月均消费频度`。

我们可以看到，月均消费频度的取值大部分在`0~75`之间，`75`以上的较少。因此接下来我们将`[0,75]`划分为5个区间，分析月均消费频度各区间上客户价值等级的取值分布。

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 设置画布大小
fig = plt.figure(figsize=(8,6))

# 绘制月均消费频度的取值分布直方图
sns.distplot(user_features['mon_consume_frq'], color='pink',kde=True)

# 设置标题
plt.title('月均消费频度取值分布')

# 设置x轴标签
plt.xlabel('月均消费频度')

plt.show()

## 5.23 客户价值等级与月均消费频度的可视化分析
观察了月均消费频度的取值分布之后，我们发现其值大多分布在0-75之间。我们将该数据在[0,75]上分为五个区间，观察在不同区间上高价值客户与低价值客户的分布情况。

对于双重分组计算频数的问题，我们还是采用交叉表的方式。在前面我们已经学习了交叉表的计算方法，使用Pandas的`crosstab()`函数。最终对两个特征进行交叉分组，再对列进行重命名，结果如下：

<center><font size="2.5px" face="微软雅黑" color="666666">表5.13：交叉表内容示例</font></center>

|  分组特征  |低价值客户  |  高价值客户|
|:-:|:-:|:-:|
|[0,  15)       |   3228    |  1073|
|[15,  30)     |   1345     |   782|
|[30,  45)     |     431      |  322|
|[45,  60)     |     131     |   119|
|[60,  75)     |       53      |    50|

## 实训任务
+ 请将月均消费频度(`mon_consume_frq`)列的数值划分到不同区间上，存储在变量`frq_range`中，此步骤题目已默认完成。
+ 请使用交叉表计算月均消费频度和客户价值等级之间的分组频数分布，保存在DataFrame对象`counts`中。
+ 请对`counts`重命名列名，分别为` "低价值客户","高价值客户"`。
+ 请调用Pandas中的`plot()`函数绘制柱状图，查看客户价值等级与月均消费频度的关系，要求设置`x`轴刻度标签旋转360度，柱形宽度`width`设为0.7。
+ 请设置`x`轴标签为`月均消费频度`，`y`轴标签为`人数`。
+ 请设置标题为`客户价值等级与月均消费频度的关系`。

通过结果我们可以看到不论客户等级，其月均消费频度都在`[0,15]`区间内占比较多。在月均消费频度较低的区间内，低价值客户占比相对较多；在月均消费频度较高的区间内，高价值客户占比逐渐提升，说明高价值客户偏向于有更多的消费需求。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 将月均消费频度取值划分到不同区间
frq_range = pd.cut(user_features['mon_consume_frq'],[0,15,30,45,60,75],right = False)

# 计算交叉表
counts = pd.crosstab(index=frq_range, columns=user_features['user_potential'])

# 重命名列名
counts.columns = ['低价值客户', '高价值客户']

fig = plt.figure(figsize = (8,6))

# 绘制柱状图
counts.plot(kind='bar', rot=360, width=0.7)

# 设置x、y轴标签
plt.xlabel('月均消费频度')
plt.ylabel('人数')

# 设置标题
plt.title('客户价值等级与月均消费频度的关系')

plt.show()

## 5.24 文本标签的提取方法
前面的步骤中，我们已经根据客户的交易记录建立了事实类标签，根据一些设定的规则建立了规则类标签，同时也建立模型预测了客户价值等级。接下来，我们要从每个客户的交易附言中提取文本标签。

在前面我们已经进行了文本预处理，具体步骤如下：
+ 中文文本分词：将中文的句子切分成有意义的词语。
+ 去除停用词：根据事先设置好的停用词，规避掉一些特殊符号或者常用但无意义的词语。

在这里我们回顾一下预处理后的`data`表。

<center><font size="2.5px" face="微软雅黑" color="666666">表5.14：data表内容示例</font></center>
<center><img src=".\Pics\Table5_14.jpg" width="650" height="125" alt="cursor" align=center /></center>

接下来我们将数据按每个客户进行分组，将每个客户的交易附言聚合在一起，保存在Series对象`describe_sum`中，以便于我们更好的提取每个客户的文本标签，`describe_sum`如表5.15所示。
<center><font size="2.5px" face="微软雅黑" color="666666">表5.15：describe_sum内容示例</font></center>
<center><img src=".\Pics\Table5_15.jpg" width="650" height="125" alt="cursor" align=center /></center>

提取文本标签(关键词)有很多种方法，在客户交易行为分析环节中，我们已经使用的Python的中文分词包`jieba`中的`analyse.extract_tags`方法尝试了一下关键词的提取，确定了标签建立的大致方向。现在我们要提取文本中的商品特征，来分析客户群体。使用Sklearn库中的文本抽取类`feature_extraction.text`，尝试进行关键词的提取，构建文本标签。

此类中有两种文本特征提取方式，分别为`CountVectorizer()`和`TfidfVectorizer()`：
+ `CountVectorizer()`是属于常见的特征数值计算类，是一个文本特征提取方法。对于每一个训练文本，它只考虑每种词汇在该训练文本中出现的频率。`CountVectorizer`会将文本中的词语转换为词频矩阵，它通过`fit_transform`函数计算各个词语出现的次数。
+  `TfidfVectorizer()`是使用`tf-idf`算法的特征数值计算类，`tf-idf`是一种统计方法，用以评估一个字词对于一个文件集或一个语料库中的其中一份文件的重要程度。字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。

最终提取的100个文本标签，组成`user_features`表的后100列，存放`TfidfVectorizer`模型提取出的文本标签的`tf-idf`值，具体的100个标签如表5.16所示。
<center><font size="2.5px" face="微软雅黑" color="666666">表5.16：data_cutted表内容示例</font></center>
<center><img src=".\Pics\Table5_16.jpg" width="650" height="125" alt="cursor" align=center /></center>

## 5.25 合并交易附言
接下来我们开始提取每个客户的文本标签，首先将数据按每个客户进行分组，然后将每个客户的交易附言文本聚合在一起。一个客户的合并流程如图5.6所示。

<center><img src=".\Pics\Pic5_8.jpg" width="650" height="125" alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图5.8 交易附言的合并流程</font></center>

按照流程合并后，最终7630个客户分词后的交易附言都融合在Series变量`describe_sum`中。
> 实训任务
> + 请将`data`按照`user_id`进行分组，将每个客户的`describe_cutted`列进行拼接，对应保存在Series变量`describe_sum`中。

In [ ]:
import pandas as pd

## 分组合并交易附言
describe_sum = data.groupby('user_id')['describe_cutted'].sum()

print(describe_sum)

## 5.26 CountVectorizer词频矩阵计算
接下来我们采用CountVectorizer模型提取词频矩阵。首先我们在原有停用词的基础上再引入一些其他停用词，将有限公司、交易、转账、消费、余额、提现等词语归于停用词，以便计算出的词能更好的反映用户真实的消费特征。停用词已预先读入，保存在变量`stop_words_list`中。

建立CountVectorizer模型的语法如下：

`sklearn.feature_extraction.text.CountVectorizer(max_df, min_df, ...)`
+ `min_df`：控制词的最小出现次数，小于`min`的词将被过滤掉。
+ `max_df`：控制词的最大出现次数，大于`max`的词将被过滤掉。
+ `max_features`：设置最终输出的关键词个数。
+ `stop_words`：设置停用词。

CountVectorizer模型建立后，可通过`fit_transform()`函数进行训练，将文本中的词语转换为词频矩阵；通过`get_feature_names()`可看到所有文本的关键字；通过`vocabulary_`属性查看关键词编号。CountVectorizer模型的输出默认为稀疏矩阵，矩阵元素`a[i][j]`表示`j`词在第`i`个文本下的词频，即各个词语出现的次数，通过`toarray()`函数可看到词频矩阵的结果。

> 实训任务
> + 请建立`CountVectorizer`模型，命名为`count_vec`，使用`CountVectorizer()`词频向量化方法，设置停用词为`stop_words_list`，设置词的最小出现次数、最大出现次数和关键词个数分别为`5, 5000, 100`。
> + 请使用`fit_transform()`的方法，对`describe_sum`计算词频矩阵，保存在变量`sparse_result_count`中，并输出稀疏矩阵、关键词及关键词编号。

+ 由于大部分文本都只会用词汇表中很少一部分的词，因此词向量中有大量的0，也就是说词向量是稀疏的。因此在实际应用中一般使用稀疏矩阵来存储，只记录非0的值和位置。
+ 在稀疏矩阵的输出中，左边的括号中的第1个数字是文本的序号i，第2个数字是词的序号j，第3个数字就是我们的词频。注意词的序号是基于所有的文档的。
+ 可以看到一共有100个词，所以每行的文本对应的都是100维的特征向量。

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# 建立CountVectorizer模型
count_vec = CountVectorizer(stop_words=stop_words_list, min_df=5, max_df=5000, max_features=100)

# 计算词频矩阵
sparse_result_count = count_vec.fit_transform(describe_sum)

# 输出稀疏矩阵
print(sparse_result_count)

# 输出关键词
print('\nvocabulary list:\n\n',count_vec.get_feature_names())

# 输出关键词编号
print('\nvocabulary dic:\n\n',count_vec.vocabulary_)


## 5.27 TfidfVectorizer词频矩阵计算
接下来我们采用TfidfVectorizer模型提取词频矩阵。建立TfidfVectorizer模型的语法如下：

`sklearn.feature_extraction.text.TfidfVectorizer(max_df, min_df,...)`
+ `min_df`：控制词的最小出现次数，小于`min`的词将被过滤掉。
+ `max_df`：控制词的最大出现次数，大于`max`的词将被过滤掉。
+ `max_features`：设置最终输出的关键词个数。
+ `stop_words`：设置停用词。

在TfidfVectorizer模型中，我们还是通过设置参数`min_df`、`max_df`来控制某个词最小/最大的出现次数，设置`max_features=100`代表最终输出的关键词为100个。和CountVectorizer模型类似，TfidfVectorizer模型也通过`get_feature_names()`查看所有文本的关键字，通过`toarray()`查看词频矩阵的结果。

> 实训任务
> + 请建立`TfidfVectorizer`模型，命名为`tfidf_vec`，使用`TfidfVectorizer()`词频向量化方法，设置停用词为`stop_words_list`，设置词的最小出现次数、最大出现次数和关键词个数分别为`5, 5000, 100`。
> + 请使用`fit_transform()`的方法，对`describe_sum`计算`tf-idf`权重矩阵，保存在变量`sparse_result_tfidf`中，并输出稀疏矩阵、关键词及关键词编号。
> + 请将提取出的100个文本标签的词频矩阵与`user_features`表进行合并，并将列名命名为各自的文本标签，此步骤题目已默认完成。

可以看到我们使用两种模型提取出的文本关键词(标签)是一样的，从模型提取出的100个文本标签中，我们可以大致描绘出如下几类人群：

<center><font size="2.5px" face="微软雅黑" color="666666">表5.17 文本标签的分类</font></center>
<center><img src=".\Pics\Table5_17.jpg" width="650" height="125" alt="cursor" align=center /></center>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 建立TfidfVectorizer模型
tfidf_vec = TfidfVectorizer(min_df=5, max_df=5000, max_features=100, stop_words=stop_words_list)

# 计算tfidf矩阵
sparse_result_tfidf = tfidf_vec.fit_transform(describe_sum)

# 输出稀疏矩阵
print(sparse_result_tfidf)

# 输出关键词
print('\nvocabulary list:\n\n',tfidf_vec.get_feature_names())

# 输出关键词编号
print( '\nvocabulary dic :\n\n',tfidf_vec.vocabulary_)

# 与user_features表进行合并
user_features = pd.concat([user_features,pd.DataFrame(sparse_result_tfidf.toarray(),index = user_features.index)],axis=1)

# 输出合并后user_features表的行列数
print('\n合并后客户标签表的行列数为:\n\n',user_features.shape)

# 重命名列名
columns_mapping = {value:key for key, value in tfidf_vec.vocabulary_.items()}
user_features.rename(columns = columns_mapping,inplace=True)

# 输出重命名后user_features表的列名
print('\n合并后客户标签表的列名为:\n\n',user_features.columns.values)

## 5.28 描绘用户画像
在提取完了客户标签之后，我们来深入分析一下客户的行为特点。我们首先选取其中交易次数(`total_transactions_cnt`)最多的客户进行分析。选出该典型客户后，我们可以通过绘制交易附言的词云图展示客户的大致画像。

创建词云对象的具体方法为：

`wordcloud.WordCloud(parameters)`，参数如下：
+ `background_color`    ：设置背景颜色。
+ `mask`    ：设置背景图案。
+ `stopwords`    ：停用词列表。
+ `font_path`    ：设置字体路径。
+ `collocations` ：是否允许词语重复，默认为`True`。
+ `scale`  ：按照比例进行放大画布，如设置为1.5，则长和宽都是原来画布的1.5倍，数值越大，产生的图片分辨率越高，字迹越清晰。
+ `random_state`  ：设置随机种子。

词云对象`wordcloud`创建好之后，使用`.generate(string)`函数，传入绘制词云的文本数据即可绘制词云。传入的文本数据为字符串格式，我们将每条交易附言进行了拼接，保存在变量`describe_document`中，可以理解为客户交易附言组成的一个文档。

> 实训任务
> + 请从`user_features`表中选取出交易次数最多的客户ID，再从`describe_sum`中选出此客户的交易附言文本，保存在字符串变量`describe_selected`中。
> + 请创建词云对象`wordcloud`，要求设置背景图案为变量`background_male`(已预先读入)，其余参数已默认给出。
> + 请根据创建的词云对象`wordcloud`，生成该客户交易附言(`describe_selected`)数据的词云图。

从词云图中可以看出，出现了很多护肤品、化妆品类的词汇，如洗面奶、控油、波斯顿等，可以大致推断客户经常接触此类商品。

In [ ]:
from wordcloud import WordCloud         
import matplotlib.pyplot as plt

# 选取文本
describe_selected = describe_sum[user_features['total_transactions_cnt'].idxmax()]

fig = plt.figure(figsize=(10,10))

# 创建词云对象
wordcloud = WordCloud(background_color="white", 
                      mask = background_male,
                      stopwords = stop_words,
                      font_path = "SimHei.ttf",
                      collocations = False,
                      scale = 4,
                      random_state = 30)

# 生成词云
wordcloud.generate(describe_selected)

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## 5.29 客户文本标签的分析
对该客户的交易流水进行可视化分析以后，我们可以对其进行文本特征的分析。在前面我们已经使用TfidfVectorizer模型提取了文本标签的tf-idf值，保存在了`user_features`表中。我们可以取出该客户文本标签的tf-idf值，将文本标签按tf-idf值由大到小进行排序，分析其行为特点。

`sort_values()`函数可以将Series对象中的元素进行排序，函数语法为：`sort_values(ascending=True)`
+ `ascending`：设置排列顺序，默认为True，代表升序排列，如果为`Fasle`则为降序排列。

> 实训任务
> + 请从`user_features`中选取出该客户(`user_id=10014615`)文本标签的tf-idf值所在的100列，保存在变量`tfidf_features`中，此步骤题目已默认完成。
> + 请选取该客户tf-idf值较大的前20个文本标签，保存在变量`top_describe`中。

- 从交易附言关键词来看，其中专柜、男士、保湿产生较多，推测该客户为一位淘宝男士护肤品卖家；卡通，儿童，宝宝这些词的出现，推测该用户是一位关注儿童用品的客户。
- 我们还可以看到另外一些关键词都与日常生活需求相关，主要是穿衣需求。

In [ ]:
import pandas as pd

# 选取存放文本标签词频的行和列
tfidf_features = user_features.loc[10014615][-100:]

# 选取出现最多的前20个交易附言关键词
top_describe = tfidf_features.sort_values(ascending=False)[:20]

print(top_describe)

## 5.30 客户数值标签的分析
在前面我们分析了该客户的文本标签，接下来可以取出该客户的其他数值标签(事实、规则和预测类)进行整体的分析。在构建了客户标签体系后，我们已经将客户的标签存储在`user_features`中。

> 实训任务
> + 请从`user_features`表中筛选出该客户(`user_id=10014615`)的数值标签，并将其存储Series对象`number_feature`中，打印该客户的数值标签，分析其特点。
>   + 提示：`user_features`表的前50列包含了事实、规则和预测类标签。

- 从其数字特征分布情况来看，其消费订单占比在33%左右，推测该客户不经常发生购买行为；
- 该客户的总体交易金额极大，转账次数极多，最常使用信用卡进行交易，信用卡还款次数与金额都相对较多，推测该客户需要使用信用卡来进行大额交易；
- 作为该平台的高端消费客户，该客户同样也使用了分期功能，推测该客户会购买价值较高的商品。

In [ ]:
import pandas as pd

# 查看该客户的数值标签
number_feature = user_features.loc[10014615][:50]

print(number_feature)

## 5.31 描绘用户画像
选取了交易流水最多的客户进行分析后，接着我们再随机选取一名客户进行用户画像的构建和分析。这里我们选取了ID为`12521063`的客户，先通过词云图展示客户的大致画像。Python词云库`wordcloud`绘制词云具体方法为：

创建词云对象的具体方法为：

`wordcloud.WordCloud(parameters)`，参数如下：
+ `background_color`    ：设置背景颜色。
+ `mask`    ：设置背景图案。
+ `stopwords`    ：停用词列表。
+ `font_path`    ：设置字体路径。
+ `collocations` ：是否允许词语重复，默认为`True`。
+ `scale`  ：按照比例进行放大画布，如设置为1.5，则长和宽都是原来画布的1.5倍，数值越大，产生的图片分辨率越高，字迹越清晰。
+ `random_state`  ：设置随机种子。

词云对象`wordcloud`创建好之后，使用`.generate(string)`函数，传入绘制词云的文本数据即可绘制词云。传入的文本数据为字符串格式，我们将每条交易附言进行了拼接，保存在变量`describe_document`中，可以理解为客户交易附言组成的一个文档。

> 实训任务
> + 请从`describe_sum`中选取出ID为`12521063`客户的交易附言，保存在字符串变量`describe_selected`中。
> + 请创建词云对象`wordcloud`，要求设置背景图案为变量`background_female`(已预先读入)，其余参数已默认给出。
> + 请根据创建的词云对象`wordcloud`，生成该客户交易附言(`describe_selected`)数据的词云图。

In [ ]:
from wordcloud import WordCloud         
import matplotlib.pyplot as plt

# 选取文本
describe_selected = describe_sum[12521063]

fig = plt.figure(figsize=(10,10))

# 创建词云对象
wordcloud = WordCloud(background_color="white", 
                      mask = background_female,
                      stopwords = stop_words,
                      font_path = "SimHei.ttf",
                      collocations = False,
                      scale = 4,
                      random_state = 30)

# 生成词云
wordcloud.generate(describe_selected)

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## 5.32 客户文本标签的分析
对该客户的交易流水进行可视化分析以后，发现该客户资金流入和流出数额均较大，推测是一个重点客户。与前一个客户相似，我们可以对其进行文本标签的分析。

在前面我们已经使用TfidfVectorizer模型提取了文本标签的tf-idf值，保存在了`user_features`表中。我们可以取出该客户的文本标签，将该客户的文本标签的tf-idf值按由大到小进行排序，分析其行为特点。

`sort_values()`函数可以将Series对象中的元素进行排序，函数语法为：`sort_values(ascending=True)`
+ `ascending`：设置排列顺序，默认为True，代表升序排列，如果为`Fasle`则为降序排列。

> 实训任务
> + 请从`user_features`中选取出该客户(`user_id=12521063`)文本标签的tf-idf值所在的100列，保存在变量`tfidf_features`中，此步骤题目已默认完成。
> + 请选取该客户tf-idf值较大的前20个交易附言关键词，保存在变量`top_describe`中。

-  从该客户的交易附言关键词可以看出，淘宝购物内容居多。从衣物到护肤品，种类丰富，推测该客户是一位网购深度“中毒”者；
- 该客户的衣物购买信息较为丰富，男士、女士、儿童均有覆盖，四季衣物均有涉猎，且其关键字中包含婴儿、儿童这一系列字样，推测该客户是刚生孩子不久的已婚女性。

In [ ]:
import pandas as pd

# 选取存放文本标签词频的行和列
tfidf_features = user_features.loc[12521063][-100:]

# 选取出现最多的前20个交易附言关键词
top_describe = tfidf_features.sort_values(ascending=False)[:20]

print(top_describe)

## 5.33 客户数值标签的分析
在前面我们分析了该客户的文本标签，接下来可以取出该客户的其他数值标签(事实、规则和预测类)进行整体的分析。在构建了客户标签体系后，我们已经将客户的标签存储在`user_features`中。

> 实训任务
> + 请从`user_features`表中筛选出该客户(`user_id=12521063`)的数值标签，并将其存储Series对象`number_feature`中，打印该客户的数值标签，分析其特点。
>   + 提示：`user_features`表的前50列包含了事实、规则和预测类标签。

- 与文本特征得出的结果相似，该客户的主要消费渠道为支付宝，其网购次数多达1300次，是淘宝的忠实用户；
- 与网购次数相反，网购金额并不是很多，平均消费60元左右，虽然并非高端消费客户，但由于其偏向网络消费，客户潜力依然较大。

In [ ]:
import pandas as pd

# 查看该客户的数字特征
number_feature = user_features.loc[12521063][:50]

print(number_feature)